In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [3]:
credentials_location = 'gcp-creds.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [5]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/03/03 17:01:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 17:01:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/03 17:01:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/03/03 17:01:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

### green data

In [11]:
df_green = spark.read.parquet('gs://spark_gcs_test_module5/pq/green/*/*')

In [13]:
df_green.createOrReplaceTempView('green')


In [15]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [17]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-06 07:00:00| 241|            169.16|             6|
|2020-01-19 19:00:00|  41| 457.2400000000003|            43|
|2020-01-23 23:00:00| 129| 304.1300000000001|            22|
|2020-01-28 15:00:00| 129|            208.23|             9|
|2020-01-02 08:00:00|  74|1187.4199999999987|            96|
|2020-01-06 16:00:00| 129|250.42000000000002|            15|
|2020-01-07 10:00:00| 171|             38.98|             2|
|2020-01-09 15:00:00| 260| 242.6800000000001|            19|
|2020-01-16 20:00:00|  82| 424.1900000000001|            27|
|2020-01-21 16:00:00|  33| 917.3599999999994|            38|
|2020-01-08 17:00:00| 140|            313.81|             8|
|2020-01-22 08:00:00| 244| 285.3999999999999|            14|
|2020-01-30 14:00:00|  74| 738.6799999999996|            49|
|2020-01-03 07:00:00|  7

In [19]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('gs://spark_gcs_test_module5/report/revenue/green', mode='overwrite')

24/03/03 17:03:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


#### yellow data

In [21]:
df_yellow = spark.read.parquet('gs://spark_gcs_test_module5/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

/Users/amohan/opt/anaconda3/envs/learn/lib/python3.9/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [22]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [25]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-25 01:00:00| 186|2287.5599999999986|           128|
|2020-01-24 07:00:00| 186| 7247.550000000012|           483|
|2020-01-04 18:00:00| 161| 7807.060000000018|           487|
|2020-01-03 12:00:00| 186| 7301.100000000017|           418|
|2020-01-22 19:00:00| 140|           4461.66|           263|
|2020-01-21 13:00:00| 162| 8327.450000000013|           491|
|2020-01-06 20:00:00| 234|5267.8600000000015|           333|
|2020-01-23 12:00:00| 233| 3545.589999999998|           184|
|2020-01-15 13:00:00| 239| 4100.529999999999|           289|
|2020-01-17 08:00:00| 170| 7410.990000000011|           475|
|2020-01-22 21:00:00|  79| 6247.700000000006|           358|
|2020-01-15 15:00:00| 239| 5217.580000000002|           344|
|2020-01-11 18:00:00| 236| 9088.440000000017|           612|
|2020-01-12 17:00:00| 23

In [27]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('gs://spark_gcs_test_module5/report/revenue/yellow', mode='overwrite')

24/03/03 17:05:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
